In [1]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import emoji
import jaconv

In [2]:
cd ../

C:\Users\井原輝人\Desktop\sorcecode\twitter-compe


In [3]:
train_df = pd.read_csv('./data/preprosessing_train.csv')

In [4]:
X_df_train = train_df[["is_url","is_location","num_emoji","num_reply","num_hash","text_length","num_kusa","is_date","is_time","text_wakati"]]
Y_df_train = train_df["flg"].astype(np.int64)
X_train =  X_df_train.values
y_train = Y_df_train.values
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.8, random_state=1)


In [5]:
X_df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20285 entries, 0 to 20284
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   is_url       20285 non-null  float64
 1   is_location  20285 non-null  float64
 2   num_emoji    20285 non-null  float64
 3   num_reply    20285 non-null  float64
 4   num_hash     20285 non-null  float64
 5   text_length  20285 non-null  float64
 6   num_kusa     20285 non-null  float64
 7   is_date      20285 non-null  float64
 8   is_time      20285 non-null  float64
 9   text_wakati  20285 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.5+ MB


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cv = CountVectorizer()
# トレーニング用データにCountVectorizerを適用
X_train_cv = cv.fit_transform(X_train[:,9])
X_train = np.block([X_train[:,:8],X_train_cv.toarray()])


# バリデーション用データにCountVectorizerを適用
X_valid_cv = cv.transform(X_valid[:,9])
X_valid = np.block([X_valid[:,:8],X_valid_cv.toarray()])


MemoryError: Unable to allocate 4.03 GiB for an array with shape (16228, 33358) and data type int64

In [ ]:
X_train.shape

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score, recall_score,f1_score
import seaborn as sns
def result_heatmap(Y_test,Y_pred):
    print("正解率:"+str(accuracy_score(Y_test, Y_pred)))
    print("適合率:"+str(recall_score(Y_test,Y_pred)))
    print("再現率:"+str(precision_score(Y_test,Y_pred)))
    print("F値:"+str(f1_score(Y_test, Y_pred)))  
    cm = confusion_matrix(Y_test, Y_pred)
    sns.heatmap(cm,annot=True, cmap='Blues')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10,
                                    random_state=1)
model.fit(X_train, y_train)

In [ ]:
#普通にやると全部違う方に分類される
y_pred = model.predict(X_valid)
result_heatmap(y_valid,y_pred)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
def draw_learning_curve(X_train,Y_train,pipe,cvnum):
    train_sizes, train_scores, test_scores =\
                    learning_curve(estimator=pipe,
                                   X=X_train,
                                   y=Y_train,
                                   train_sizes=np.linspace(0.1, 1.0, cv_num),
                                   cv=cvnum,
                                   n_jobs=1)#0.1~1の10刻み（割合になっている）

    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    plt.plot(train_sizes, train_mean,
             color='blue', marker='o',
             markersize=5, label='training accuracy')

    plt.fill_between(train_sizes,
                     train_mean + train_std,
                     train_mean - train_std,
                     alpha=0.15, color='blue')

    plt.plot(train_sizes, test_mean,
             color='green', linestyle='--',
             marker='s', markersize=5,
             label='validation accuracy')

    plt.fill_between(train_sizes,
                     test_mean + test_std,
                     test_mean - test_std,
                     alpha=0.15, color='green')

    plt.grid()
    plt.xlabel('Number of training samples')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.ylim([0.8, 1.03])
    plt.tight_layout()
    plt.show()